# Importações

In [36]:
from selenium import webdriver
import mysql.connector
from mysql.connector import errorcode


# Funções

### conexão MySQL

In [37]:
def conecta():
    try:
        global conexao
        conexao = mysql.connector.connect(
          host="localhost", #ip
          user="root", #usuario
          password="123456", #senha
          database="inpi" #banco
        )
        global cursor
        cursor = conexao.cursor()

    except mysql.connector.Error as error:
        if error.errno == errorcode.ER_BAD_DB_ERROR:
            print("Banco de dados não existe")
        elif error.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Usuario ou senha incorretos")
        else:
            print(error)
        return False
    else:
        print("Conexão realizada com sucesso")
        return True

### Recupera uma faixa para trabalhar
A = Aberta /
P = Processando /
F = Finalizada /
M = Em manutenção

In [38]:
def faixa():
    sql = "select Id, Inicio, Fim, Progresso from tb_faixas where Status = 'A' LIMIT 1"
    cursor.execute(sql)
    fx = cursor.fetchone()

    global idFaixa
    idFaixa = fx[0]
    global inicio
    inicio = fx[1]
    global total
    total = fx[2]
    global progresso
    progresso = fx[3]


### Atualiza faixa para não haver conflitos

In [39]:
def attFaixa():
    sql = "update tb_faixas set Status = 'P' where Id = " + str(idFaixa)
    cursor.execute(sql)

    conexao.commit()


### Configura driver e inicia

In [40]:
def browser():
    options = webdriver.ChromeOptions()
    options.add_argument('ignore-certificate-errors')
    global driver
    driver = webdriver.Chrome(chrome_options=options)


### Inicia processo de autenticação para inicio da mineração


In [41]:
def autentica():
    driver.get("https://gru.inpi.gov.br/pePI/servlet/LoginController?action=login")
    driver.get("https://gru.inpi.gov.br/pePI/jsp/marcas/Pesquisa_classe_basica.jsp")
    input_marca = driver.find_element_by_name("marca")
    input_marca.send_keys("teste")

    submit_button = driver.find_element_by_name("botao")
    submit_button.click()

# Inicio Mineração

In [ ]:
if conecta():

    faixa()
    attFaixa()
    browser()
    autentica()

    for n in range(progresso,total):
        uri = "https://gru.inpi.gov.br/pePI/servlet/MarcasServletController?Action=detail&CodPedido=" + str(n)
        driver.get(uri)

#

Conexão realizada com sucesso


<ipython-input-40-fe5f3451e6d1>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options)
